In [1]:
import numpy as np
import os
import xarray as xr
import glob
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.colors import BoundaryNorm
import matplotlib.colors as mcolors
import dask.array as da
import pickle
from scipy.stats import t
from scipy.interpolate import interp1d
from metpy.units import units

In [2]:
# reading in ERA5 u, v, w and q

base_path = '/data/rong4/Data/ERA5/3hourly/quvw_US'

years = [str(year) for year in range(2001, 2019)]

def get_files(folder, component):

    files = glob.glob(os.path.join(base_path, folder, f"era5.{component}.*.nc"))

    filtered_files = [f for f in files if any(year in f for year in years)]
    
    return filtered_files

# Get the files for each component

u_files = get_files('u_component_of_wind', 'u_component_of_wind')

v_files = get_files('v_component_of_wind', 'v_component_of_wind')

w_files = get_files('vertical_velocity', 'vertical_velocity')

q_files = get_files('specific_humidity', 'specific_humidity')

all_files = u_files + v_files + q_files + w_files

# open all datasets at once

uvwq = xr.open_mfdataset(all_files, combine='by_coords', chunks={'time': 24})

# filtering for correct domain and time

uvwq = uvwq.sel(
    
    latitude=slice(39, 30),
    
    longitude=slice(-105, -95)
)

uvwq['time'] = uvwq['time'] - pd.Timedelta(hours=6)

sub = uvwq.where(
    
    (uvwq.time.dt.hour.isin([0, 6, 12])) &
    
    (uvwq.time.dt.month.isin([5, 6, 7, 8, 9])) &
    
    (uvwq.time.dt.year.isin(range(2001, 2019))),
    
    drop=True
)

# Resample to daily frequency and compute the mean

uvwq = sub.resample(time='D').mean().dropna(dim='time', how='all')

uvwq['time'].shape

(2754,)

In [4]:
# ERA5 information: surface pressure

sfc_pressure_path = '/data/rong4/Data/ERA5/3hourly/quvw_US/surface_pressure'

all_files = glob.glob(os.path.join(sfc_pressure_path, "era5.surface_pressure.*.nc"))

# Filter files for years 2001-2018 and months 5-9

filtered_files = [
    
    file for file in all_files 
    
    if 2001 <= int(os.path.basename(file).split(".")[2][:4]) <= 2018  
    
    and 5 <= int(os.path.basename(file).split(".")[2][4:]) <= 9       
]

all_sfc_p = xr.open_mfdataset(filtered_files, combine='by_coords')

all_sfc_p = all_sfc_p.sel(latitude=slice(39.0, 30.0), longitude=slice(-105.0, -95.0))

sfcp = all_sfc_p.assign_coords(time=all_sfc_p.time - pd.Timedelta(hours=6))

sfcp = sfcp.sel(time=sfcp.time[sfcp.time.dt.hour == 12])

sfcp['time'].shape

(2754,)

In [5]:
# ERA5 geopotential height & temperature

tz_path = '/home/annierosen16/data/era5sgp/era5_ml_t_geopotential_full_sgp'

file_pattern = f"{tz_path}/ERA5_*"

file_list = glob.glob(file_pattern)

all_tz = xr.open_mfdataset(file_list, combine='by_coords', chunks={'time': 24})

# all_tz = all_tz.sel(pressure_level=(all_tz.pressure_level >= 450) & (all_tz.pressure_level <= 1000))

all_tz = all_tz.assign_coords(valid_time=all_tz.valid_time - pd.Timedelta(hours=6))

tzdf = all_tz.sel(valid_time=all_tz.valid_time[all_tz.valid_time.dt.hour == 12])

tzdf = tzdf.sel(pressure_level=tzdf.pressure_level >= 50)

tzdf['valid_time'].shape, tzdf['pressure_level'].data.shape

((2754,), (29,))

In [3]:
# loading all necessary information for advection 

g = 9.80665 # m/s^2

# 1 dimensional

times = uvwq['time'].data

lats = uvwq['latitude'].data

lons = uvwq['longitude'].data

# 4 dimensional

q = uvwq['q'].load().data 

print("q loaded")

u = uvwq['u'].load().data

print("u loaded")

v = uvwq['v'].load().data

print("v loaded")

w = uvwq['w'].load().data

print("w loaded")

# z = tzdf['z'].load().data / g

# print("z loaded")

# # 3 dimensional (time, lat, lon)

# sp = sfcp['sp'].load().data 

# print("sp loaded")

q loaded
u loaded
v loaded
w loaded


In [47]:
## reversing arrays to go from bottom to top for interpolation

u = u[:, ::-1, :, :]

v = v[:, ::-1, :, :]

w = w[:, ::-1, :, :]

q = q[:, ::-1, :, :]

z = z[:, ::-1, :, :]

In [9]:
# ## reversing latitude for advection 

# old_uflip = u[:, :, ::-1, :]

# old_vflip = v[:, :, ::-1, :]

# old_wflip = w[:, :, ::-1, :]

# old_qflip = q[:, :, ::-1, :]

# old_latsflip = lats[::-1]

In [8]:
## load interpolated u, v, w and q: 

# Step 1: Define the backup directory in your home folder

backup_dir = os.path.expanduser("~/backup_data")  # Expands ~ to full home directory path

file_path = os.path.join(backup_dir, "backup_arrays_for_advection.npz")  # Full path to save file

# Step 5: Load back the saved arrays

data = np.load(file_path)  # Load the .npz file

times=data['times']

lats=data['lats']

lons=data['lons']

interpolated_alt=data['interpolated_alt']

interpolated_pressure = data['interpolated_pressure']

print("interpolated_pressure loaded")

interpolated_q = data['interpolated_q']

print("interpolated_q loaded")

interpolated_u = data['interpolated_u']

print("interpolated_u loaded")

interpolated_v = data['interpolated_v']

print("interpolated_v loaded")

interpolated_w = data['interpolated_w']

print("Arrays successfully loaded!")

interpolated_pressure loaded
interpolated_q loaded
interpolated_u loaded
interpolated_v loaded
Arrays successfully loaded!


In [19]:
# # flipping arrays for comparison advection calculation (top to bottom and latitudes increasing)

# interp_p_flip = interpolated_pressure[:, ::-1, ::-1, :] * 100

# interp_u_flip = interpolated_u[:, ::-1, ::-1, :]

# interp_v_flip = interpolated_v[:, ::-1, ::-1, :]

# interp_w_flip = interpolated_w[:, ::-1, ::-1, :]

# interp_q_flip = interpolated_q[:, ::-1, ::-1, :]

In [13]:
p1 = tzdf['pressure_level'].data[::-1] * 100

tidx = 0

latidx = 0

lonidx = 0

q1 = q[tidx, :, latidx, lonidx]

q2 = interpolated_q[tidx, :, latidx, lonidx]

p2 = interpolated_pressure[tidx, :, latidx, lonidx] * 100

p3 = p_flip[tidx, :, -1, 0][::-1]

q3 = q_flip[tidx, :, -1, 0][::-1]

plt.plot(q1, p1, color='blue', label='old')

plt.scatter(q2, p2, color='orange', label='new')

plt.scatter(q2, p2, color='green', label='flip')

plt.legend()

plt.gca().invert_yaxis()

plt.show()

In [63]:
# # process 1

# lat = 33 

# lon = -100

# tidx = 0

# print(lat, lon, times[tidx])

# lat_idx = np.where(lats == lat)[0][0]

# lon_idx = np.where(lons == lon)[0][0]

# # Define the center point and half-window (in degrees)

# delta = 0.5

# lat_mask = (old_latsflip >= (lat - delta)) & (old_latsflip <= (lat + delta))

# lon_mask = (lons >= (lon - delta)) & (lons <= (lon + delta))

# lat_indices = np.where(lat_mask)[0]

# lon_indices = np.where(lon_mask)[0]

# lat_mini = old_latsflip[lat_indices]

# lon_mini = lons[lon_indices]

# # select specific time, lat and lon (and surrounding boundary) for every APE

# # these all have shapes (pressure_levels, mini_lat, mini_lon) (29, 5, 5)

# p1 = uvwq["level"].data * 100

# u_mini = old_uflip[tidx, :, :, :]

# u_mini = u_mini[:, lat_mask, :][:, :, lon_mask]

# v_mini = old_vflip[tidx, :, :, :]

# v_mini = v_mini[:, lat_mask, :][:, :, lon_mask]

# q_mini = old_qflip[tidx, :, :, :]

# q_mini = q_mini[:, lat_mask, :][:, :, lon_mask]

# w_mini = old_wflip[tidx, :, :, :]

# w_mini = w_mini[:, lat_mask, :][:, :, lon_mask]

# # horizontal advection

# zonal_adv1, merid_adv1 = calculate_horizontal_advection(u_mini, v_mini, q_mini, lat_mini, lon_mini)

# # select specific time, lat and lon (and surrounding boundary) for every APE

# # these all have shapes (pressure_levels, mini_lat, mini_lon) (29, 5, 5)

# p_mini = interp_p_flip[tidx, :, :, :]

# p_mini = p_mini[:, lat_mask, :][:, :, lon_mask]

# u_mini = interp_u_flip[tidx, :, :, :]

# u_mini = u_mini[:, lat_mask, :][:, :, lon_mask]

# v_mini = interp_v_flip[tidx, :, :, :]

# v_mini = v_mini[:, lat_mask, :][:, :, lon_mask]

# q_mini = interp_q_flip[tidx, :, :, :]

# q_mini = q_mini[:, lat_mask, :][:, :, lon_mask]

# w_mini = interp_w_flip[tidx, :, :, :]

# w_mini = w_mini[:, lat_mask, :][:, :, lon_mask]

# # horizontal advection

# zonal_adv2, merid_adv2 = calculate_horizontal_advection(u_mini, v_mini, q_mini, lat_mini, lon_mini)

# print(lat_mini, lon_mini)


In [64]:
# # plt.plot(zonal_adv[::-1], p_mini[:, 1, 1][::-1], label='zonal')

# p2 = p_mini[:, 2, 2]

# plt.plot(merid_adv2, p2, label='interp')

# plt.plot(merid_adv1, p1, label='original')

# # plt.ylim(80000, 50000)

# # plt.xlim(-0.5, 0.2)

# plt.gca().invert_yaxis()

# plt.legend()

# plt.grid()

# plt.show()

In [65]:
# def calculate_horizontal_advection(u, v, q, lat, lon):

#     R = 6371000  # radius of Earth (meters)
    
#     rho_water = 1000 # kg/m^3
    
#     seconds_to_day = 60 * 60 * 24 # to convert from s --> day

#     latitude_range = np.sin(np.radians(lat[-1])) - np.sin(np.radians(lat[0]))
    
#     longitude_range = np.radians(lon[-1]) - np.radians(lon[0])
    
#     lon_grid, lat_grid = np.meshgrid(np.radians(lon), np.radians(lat))
    
#     delta_y = np.radians(lat[1]) - np.radians(lat[0])
        
#     delta_x = np.radians(lon[1]) - np.radians(lon[0])
        
#     # shaped as (#lons, #lats) so first row represents all values for first latitude (eg: 30,30,30...)
#     cos_correction = np.cos(lat_grid)
    
#     dqdx = np.zeros_like(q)
    
#     dqdy = np.zeros_like(q)
    
#     # Loop through each pressure level
    
#     for k in range(u.shape[0]):
        
#         # Central points
    
#         dqdx[k, :, 1:-1] = (q[k, :, 2:] - q[k, :, :-2]) / (2 * delta_x)
#         dqdy[k, 1:-1, :] = (q[k, 2:, :] - q[k, :-2, :]) / (2 * delta_y)
        
#         # Left and right boundaries (x-direction)
        
#         dqdx[k, :, 0] = (q[k, :, 1] - q[k, :, 0]) / (delta_x)
#         dqdx[k, :, -1] = (q[k, :, -2] - q[k, :, -1]) / (delta_x)

#         # Top and bottom boundaries (y-direction)
        
#         dqdy[k, 0, :] = (q[k, 1, :] - q[k, 0, :]) / (delta_y)
#         dqdy[k, -1, :] = (q[k, -2, :] - q[k, -1, :]) / (delta_y)
        
#         dx = R * cos_correction * delta_x
#         dy = R * delta_y

#         # for every grid point, multiply by the area extent of that grid box 
        
#         dqdy[k] = (1/R) * dqdy[k] * dx * dy
        
#         dqdx[k] = (1/(R*cos_correction)) * dqdx[k] * dx * dy
        
#     meridional_advection = -(v * dqdy)
    
#     zonal_advection = -(u * dqdx)
    
#     total_area = []
    
#     for i in dx: 
        
#         total_area.append(i*dy)
        
#     A = np.sum(total_area)
        
#     # Area weighted average
#     zonal_advection_area_weighted = np.sum(zonal_advection, axis=(1,2))*(1/A)*seconds_to_day*rho_water
    
#     meridional_advection_area_weighted = np.sum(meridional_advection, axis=(1,2))*(1/A)*seconds_to_day*rho_water

#     # Units = 10^3/day 
#     # Shapes returned: (#pressure levels) because advection is averaged over the domain for every level 
    
#     return zonal_advection_area_weighted, meridional_advection_area_weighted

In [113]:
# step 1 -> interpolate pressure to altitude

def interpolate_pressure_to_altitude(pres_array, alt_array, surface_pressure):
    
    x = alt_array
    
    y = pres_array

    f = interp1d(x, y, assume_sorted=False, fill_value="extrapolate")

    max_x = np.nanmax(x)
    
    min_x = np.nanmin(x)

    xint = np.arange(min_x, max_x, 100)

    # Interpolate pressure against new altitude intervals
    
    yint = f(xint) * units.hPa
    
    yint = yint.magnitude  # Extract numerical values

    # Compute altitude above surface
    
    mask = yint <= surface_pressure
    
    xnew = xint[mask]
    
    ynew = yint[mask]

    if xnew.size == 0:
        
        return np.array([]), np.array([]), np.nan 

    sfc_alt = xnew[0]  # sfc_alt is the height of the surface above sea-level
    
    xnew_rel = xnew - sfc_alt # shift altitude heights to be starting at 0

    # Apply single mask to keep values within 4000m AGL
    
    valid_mask = xnew_rel <= 4000 # meters AGL
    
    xclean = xnew_rel[valid_mask]
    
    yclean = ynew[valid_mask]

    return yclean, sfc_alt

In [124]:
# for latitudes & longitudes: interoplate pressure 

num_levels = 41 # equivalent to 4000 meters AGL for every gridpoint

num_times = times.shape[0]

# from bottom to top 

pres_array = tzdf['pressure_level'].data[::-1]

interpolated_pressure = np.full((num_times, num_levels, lats.shape[0], lons.shape[0]), np.nan)

# surface_altitudes captures geopotential height at the surface for all times and gridpoints

surface_altitudes = np.full((num_times, lats.shape[0], lons.shape[0]), np.nan)

for tidx in range(times.shape[0]):
    
    print(times[tidx])
    
    for latidx in range(lats.shape[0]):

        for lonidx in range(lons.shape[0]):

            alt_array = z[tidx, :, latidx, lonidx]

            sfc_pres = sp[tidx, latidx, lonidx] / 100

            new_pres, sfc_alt = interpolate_pressure_to_altitude(pres_array, alt_array, sfc_pres)
        
            interpolated_pressure[tidx, :, latidx, lonidx] = new_pres

            surface_altitudes[tidx, latidx, lonidx] = sfc_alt

2001-05-01T00:00:00.000000000
2001-05-02T00:00:00.000000000
2001-05-03T00:00:00.000000000
2001-05-04T00:00:00.000000000
2001-05-05T00:00:00.000000000
2001-05-06T00:00:00.000000000
2001-05-07T00:00:00.000000000
2001-05-08T00:00:00.000000000
2001-05-09T00:00:00.000000000
2001-05-10T00:00:00.000000000
2001-05-11T00:00:00.000000000
2001-05-12T00:00:00.000000000
2001-05-13T00:00:00.000000000
2001-05-14T00:00:00.000000000
2001-05-15T00:00:00.000000000
2001-05-16T00:00:00.000000000
2001-05-17T00:00:00.000000000
2001-05-18T00:00:00.000000000
2001-05-19T00:00:00.000000000
2001-05-20T00:00:00.000000000
2001-05-21T00:00:00.000000000
2001-05-22T00:00:00.000000000
2001-05-23T00:00:00.000000000
2001-05-24T00:00:00.000000000
2001-05-25T00:00:00.000000000
2001-05-26T00:00:00.000000000
2001-05-27T00:00:00.000000000
2001-05-28T00:00:00.000000000
2001-05-29T00:00:00.000000000
2001-05-30T00:00:00.000000000
2001-05-31T00:00:00.000000000
2001-06-01T00:00:00.000000000
2001-06-02T00:00:00.000000000
2001-06-03

2002-08-30T00:00:00.000000000
2002-08-31T00:00:00.000000000
2002-09-01T00:00:00.000000000
2002-09-02T00:00:00.000000000
2002-09-03T00:00:00.000000000
2002-09-04T00:00:00.000000000
2002-09-05T00:00:00.000000000
2002-09-06T00:00:00.000000000
2002-09-07T00:00:00.000000000
2002-09-08T00:00:00.000000000
2002-09-09T00:00:00.000000000
2002-09-10T00:00:00.000000000
2002-09-11T00:00:00.000000000
2002-09-12T00:00:00.000000000
2002-09-13T00:00:00.000000000
2002-09-14T00:00:00.000000000
2002-09-15T00:00:00.000000000
2002-09-16T00:00:00.000000000
2002-09-17T00:00:00.000000000
2002-09-18T00:00:00.000000000
2002-09-19T00:00:00.000000000
2002-09-20T00:00:00.000000000
2002-09-21T00:00:00.000000000
2002-09-22T00:00:00.000000000
2002-09-23T00:00:00.000000000
2002-09-24T00:00:00.000000000
2002-09-25T00:00:00.000000000
2002-09-26T00:00:00.000000000
2002-09-27T00:00:00.000000000
2002-09-28T00:00:00.000000000
2002-09-29T00:00:00.000000000
2002-09-30T00:00:00.000000000
2003-05-01T00:00:00.000000000
2003-05-02

2004-07-29T00:00:00.000000000
2004-07-30T00:00:00.000000000
2004-07-31T00:00:00.000000000
2004-08-01T00:00:00.000000000
2004-08-02T00:00:00.000000000
2004-08-03T00:00:00.000000000
2004-08-04T00:00:00.000000000
2004-08-05T00:00:00.000000000
2004-08-06T00:00:00.000000000
2004-08-07T00:00:00.000000000
2004-08-08T00:00:00.000000000
2004-08-09T00:00:00.000000000
2004-08-10T00:00:00.000000000
2004-08-11T00:00:00.000000000
2004-08-12T00:00:00.000000000
2004-08-13T00:00:00.000000000
2004-08-14T00:00:00.000000000
2004-08-15T00:00:00.000000000
2004-08-16T00:00:00.000000000
2004-08-17T00:00:00.000000000
2004-08-18T00:00:00.000000000
2004-08-19T00:00:00.000000000
2004-08-20T00:00:00.000000000
2004-08-21T00:00:00.000000000
2004-08-22T00:00:00.000000000
2004-08-23T00:00:00.000000000
2004-08-24T00:00:00.000000000
2004-08-25T00:00:00.000000000
2004-08-26T00:00:00.000000000
2004-08-27T00:00:00.000000000
2004-08-28T00:00:00.000000000
2004-08-29T00:00:00.000000000
2004-08-30T00:00:00.000000000
2004-08-31

2006-06-27T00:00:00.000000000
2006-06-28T00:00:00.000000000
2006-06-29T00:00:00.000000000
2006-06-30T00:00:00.000000000
2006-07-01T00:00:00.000000000
2006-07-02T00:00:00.000000000
2006-07-03T00:00:00.000000000
2006-07-04T00:00:00.000000000
2006-07-05T00:00:00.000000000
2006-07-06T00:00:00.000000000
2006-07-07T00:00:00.000000000
2006-07-08T00:00:00.000000000
2006-07-09T00:00:00.000000000
2006-07-10T00:00:00.000000000
2006-07-11T00:00:00.000000000
2006-07-12T00:00:00.000000000
2006-07-13T00:00:00.000000000
2006-07-14T00:00:00.000000000
2006-07-15T00:00:00.000000000
2006-07-16T00:00:00.000000000
2006-07-17T00:00:00.000000000
2006-07-18T00:00:00.000000000
2006-07-19T00:00:00.000000000
2006-07-20T00:00:00.000000000
2006-07-21T00:00:00.000000000
2006-07-22T00:00:00.000000000
2006-07-23T00:00:00.000000000
2006-07-24T00:00:00.000000000
2006-07-25T00:00:00.000000000
2006-07-26T00:00:00.000000000
2006-07-27T00:00:00.000000000
2006-07-28T00:00:00.000000000
2006-07-29T00:00:00.000000000
2006-07-30

2008-05-26T00:00:00.000000000
2008-05-27T00:00:00.000000000
2008-05-28T00:00:00.000000000
2008-05-29T00:00:00.000000000
2008-05-30T00:00:00.000000000
2008-05-31T00:00:00.000000000
2008-06-01T00:00:00.000000000
2008-06-02T00:00:00.000000000
2008-06-03T00:00:00.000000000
2008-06-04T00:00:00.000000000
2008-06-05T00:00:00.000000000
2008-06-06T00:00:00.000000000
2008-06-07T00:00:00.000000000
2008-06-08T00:00:00.000000000
2008-06-09T00:00:00.000000000
2008-06-10T00:00:00.000000000
2008-06-11T00:00:00.000000000
2008-06-12T00:00:00.000000000
2008-06-13T00:00:00.000000000
2008-06-14T00:00:00.000000000
2008-06-15T00:00:00.000000000
2008-06-16T00:00:00.000000000
2008-06-17T00:00:00.000000000
2008-06-18T00:00:00.000000000
2008-06-19T00:00:00.000000000
2008-06-20T00:00:00.000000000
2008-06-21T00:00:00.000000000
2008-06-22T00:00:00.000000000
2008-06-23T00:00:00.000000000
2008-06-24T00:00:00.000000000
2008-06-25T00:00:00.000000000
2008-06-26T00:00:00.000000000
2008-06-27T00:00:00.000000000
2008-06-28

...

2011-08-23T00:00:00.000000000
2011-08-24T00:00:00.000000000
2011-08-25T00:00:00.000000000
2011-08-26T00:00:00.000000000
2011-08-27T00:00:00.000000000
2011-08-28T00:00:00.000000000
2011-08-29T00:00:00.000000000
2011-08-30T00:00:00.000000000
2011-08-31T00:00:00.000000000
2011-09-01T00:00:00.000000000
2011-09-02T00:00:00.000000000
2011-09-03T00:00:00.000000000
2011-09-04T00:00:00.000000000
2011-09-05T00:00:00.000000000
2011-09-06T00:00:00.000000000
2011-09-07T00:00:00.000000000
2011-09-08T00:00:00.000000000
2011-09-09T00:00:00.000000000
2011-09-10T00:00:00.000000000
2011-09-11T00:00:00.000000000
2011-09-12T00:00:00.000000000
2011-09-13T00:00:00.000000000
2011-09-14T00:00:00.000000000
2011-09-15T00:00:00.000000000
2011-09-16T00:00:00.000000000
2011-09-17T00:00:00.000000000
2011-09-18T00:00:00.000000000
2011-09-19T00:00:00.000000000
2011-09-20T00:00:00.000000000
2011-09-21T00:00:00.000000000
2011-09-22T00:00:00.000000000
2011-09-23T00:00:00.000000000
2011-09-24T00:00:00.000000000
2011-09-25

2013-07-22T00:00:00.000000000
2013-07-23T00:00:00.000000000
2013-07-24T00:00:00.000000000
2013-07-25T00:00:00.000000000
2013-07-26T00:00:00.000000000
2013-07-27T00:00:00.000000000
2013-07-28T00:00:00.000000000
2013-07-29T00:00:00.000000000
2013-07-30T00:00:00.000000000
2013-07-31T00:00:00.000000000
2013-08-01T00:00:00.000000000
2013-08-02T00:00:00.000000000
2013-08-03T00:00:00.000000000
2013-08-04T00:00:00.000000000
2013-08-05T00:00:00.000000000
2013-08-06T00:00:00.000000000
2013-08-07T00:00:00.000000000
2013-08-08T00:00:00.000000000
2013-08-09T00:00:00.000000000
2013-08-10T00:00:00.000000000
2013-08-11T00:00:00.000000000
2013-08-12T00:00:00.000000000
2013-08-13T00:00:00.000000000
2013-08-14T00:00:00.000000000
2013-08-15T00:00:00.000000000
2013-08-16T00:00:00.000000000
2013-08-17T00:00:00.000000000
2013-08-18T00:00:00.000000000
2013-08-19T00:00:00.000000000
2013-08-20T00:00:00.000000000
2013-08-21T00:00:00.000000000
2013-08-22T00:00:00.000000000
2013-08-23T00:00:00.000000000
2013-08-24

2015-06-20T00:00:00.000000000
2015-06-21T00:00:00.000000000
2015-06-22T00:00:00.000000000
2015-06-23T00:00:00.000000000
2015-06-24T00:00:00.000000000
2015-06-25T00:00:00.000000000
2015-06-26T00:00:00.000000000
2015-06-27T00:00:00.000000000
2015-06-28T00:00:00.000000000
2015-06-29T00:00:00.000000000
2015-06-30T00:00:00.000000000
2015-07-01T00:00:00.000000000
2015-07-02T00:00:00.000000000
2015-07-03T00:00:00.000000000
2015-07-04T00:00:00.000000000
2015-07-05T00:00:00.000000000
2015-07-06T00:00:00.000000000
2015-07-07T00:00:00.000000000
2015-07-08T00:00:00.000000000
2015-07-09T00:00:00.000000000
2015-07-10T00:00:00.000000000
2015-07-11T00:00:00.000000000
2015-07-12T00:00:00.000000000
2015-07-13T00:00:00.000000000
2015-07-14T00:00:00.000000000
2015-07-15T00:00:00.000000000
2015-07-16T00:00:00.000000000
2015-07-17T00:00:00.000000000
2015-07-18T00:00:00.000000000
2015-07-19T00:00:00.000000000
2015-07-20T00:00:00.000000000
2015-07-21T00:00:00.000000000
2015-07-22T00:00:00.000000000
2015-07-23

2017-05-19T00:00:00.000000000
2017-05-20T00:00:00.000000000
2017-05-21T00:00:00.000000000
2017-05-22T00:00:00.000000000
2017-05-23T00:00:00.000000000
2017-05-24T00:00:00.000000000
2017-05-25T00:00:00.000000000
2017-05-26T00:00:00.000000000
2017-05-27T00:00:00.000000000
2017-05-28T00:00:00.000000000
2017-05-29T00:00:00.000000000
2017-05-30T00:00:00.000000000
2017-05-31T00:00:00.000000000
2017-06-01T00:00:00.000000000
2017-06-02T00:00:00.000000000
2017-06-03T00:00:00.000000000
2017-06-04T00:00:00.000000000
2017-06-05T00:00:00.000000000
2017-06-06T00:00:00.000000000
2017-06-07T00:00:00.000000000
2017-06-08T00:00:00.000000000
2017-06-09T00:00:00.000000000
2017-06-10T00:00:00.000000000
2017-06-11T00:00:00.000000000
2017-06-12T00:00:00.000000000
2017-06-13T00:00:00.000000000
2017-06-14T00:00:00.000000000
2017-06-15T00:00:00.000000000
2017-06-16T00:00:00.000000000
2017-06-17T00:00:00.000000000
2017-06-18T00:00:00.000000000
2017-06-19T00:00:00.000000000
2017-06-20T00:00:00.000000000
2017-06-21

2018-09-17T00:00:00.000000000
2018-09-18T00:00:00.000000000
2018-09-19T00:00:00.000000000
2018-09-20T00:00:00.000000000
2018-09-21T00:00:00.000000000
2018-09-22T00:00:00.000000000
2018-09-23T00:00:00.000000000
2018-09-24T00:00:00.000000000
2018-09-25T00:00:00.000000000
2018-09-26T00:00:00.000000000
2018-09-27T00:00:00.000000000
2018-09-28T00:00:00.000000000
2018-09-29T00:00:00.000000000
2018-09-30T00:00:00.000000000


In [132]:
# cleaner version

def interpolate_var_to_altitude(variable_to_interpolate, alt_array, sfc_alt):
    
    x = alt_array
    
    y = variable_to_interpolate

    f = interp1d(x, y, assume_sorted=False, fill_value="extrapolate")

    max_x = np.nanmax(x)
    
    min_x = np.nanmin(x)

    xnew = np.arange(min_x, max_x, 100)
    
    ynew = f(xnew)

    # Compute relative altitude
    
    xnew_rel = xnew - sfc_alt
    
    valid_mask = np.logical_and(xnew_rel >= 0, xnew_rel <= 4000)

    yfinal = ynew[valid_mask]
    
    xfinal = xnew_rel[valid_mask]

    return yfinal

In [145]:
# interpolation of specific humidity 

num_levels = 41 # equivalent to 4000 meters AGL for every gridpoint

num_times = times.shape[0]

interpolated_q = np.full((num_times, num_levels, lats.shape[0], lons.shape[0]), np.nan)

for tidx in range(times.shape[0]):
    
    print(times[tidx])

    for latidx in range(lats.shape[0]):

        for lonidx in range(lons.shape[0]):
            
            q_array = q[tidx, :, latidx, lonidx]

            alt_array = z[tidx, :, latidx, lonidx]

            sfc_alt = surface_altitudes[tidx, latidx, lonidx]

            new_q = interpolate_var_to_altitude(q_array, alt_array, sfc_alt)

            interpolated_q[tidx, :, latidx, lonidx] = new_q

2001-05-01T00:00:00.000000000
2001-05-02T00:00:00.000000000
2001-05-03T00:00:00.000000000
2001-05-04T00:00:00.000000000
2001-05-05T00:00:00.000000000
2001-05-06T00:00:00.000000000
2001-05-07T00:00:00.000000000
2001-05-08T00:00:00.000000000
2001-05-09T00:00:00.000000000
2001-05-10T00:00:00.000000000
2001-05-11T00:00:00.000000000
2001-05-12T00:00:00.000000000
2001-05-13T00:00:00.000000000
2001-05-14T00:00:00.000000000
2001-05-15T00:00:00.000000000
2001-05-16T00:00:00.000000000
2001-05-17T00:00:00.000000000
2001-05-18T00:00:00.000000000
2001-05-19T00:00:00.000000000
2001-05-20T00:00:00.000000000
2001-05-21T00:00:00.000000000
2001-05-22T00:00:00.000000000
2001-05-23T00:00:00.000000000
2001-05-24T00:00:00.000000000
2001-05-25T00:00:00.000000000
2001-05-26T00:00:00.000000000
2001-05-27T00:00:00.000000000
2001-05-28T00:00:00.000000000
2001-05-29T00:00:00.000000000
2001-05-30T00:00:00.000000000
2001-05-31T00:00:00.000000000
2001-06-01T00:00:00.000000000
2001-06-02T00:00:00.000000000
2001-06-03

2002-08-30T00:00:00.000000000
2002-08-31T00:00:00.000000000
2002-09-01T00:00:00.000000000
2002-09-02T00:00:00.000000000
2002-09-03T00:00:00.000000000
2002-09-04T00:00:00.000000000
2002-09-05T00:00:00.000000000
2002-09-06T00:00:00.000000000
2002-09-07T00:00:00.000000000
2002-09-08T00:00:00.000000000
2002-09-09T00:00:00.000000000
2002-09-10T00:00:00.000000000
2002-09-11T00:00:00.000000000
2002-09-12T00:00:00.000000000
2002-09-13T00:00:00.000000000
2002-09-14T00:00:00.000000000
2002-09-15T00:00:00.000000000
2002-09-16T00:00:00.000000000
2002-09-17T00:00:00.000000000
2002-09-18T00:00:00.000000000
2002-09-19T00:00:00.000000000
2002-09-20T00:00:00.000000000
2002-09-21T00:00:00.000000000
2002-09-22T00:00:00.000000000
2002-09-23T00:00:00.000000000
2002-09-24T00:00:00.000000000
2002-09-25T00:00:00.000000000
2002-09-26T00:00:00.000000000
2002-09-27T00:00:00.000000000
2002-09-28T00:00:00.000000000
2002-09-29T00:00:00.000000000
2002-09-30T00:00:00.000000000
2003-05-01T00:00:00.000000000
2003-05-02

2004-07-29T00:00:00.000000000
2004-07-30T00:00:00.000000000
2004-07-31T00:00:00.000000000
2004-08-01T00:00:00.000000000
2004-08-02T00:00:00.000000000
2004-08-03T00:00:00.000000000
2004-08-04T00:00:00.000000000
2004-08-05T00:00:00.000000000
2004-08-06T00:00:00.000000000
2004-08-07T00:00:00.000000000
2004-08-08T00:00:00.000000000
2004-08-09T00:00:00.000000000
2004-08-10T00:00:00.000000000
2004-08-11T00:00:00.000000000
2004-08-12T00:00:00.000000000
2004-08-13T00:00:00.000000000
2004-08-14T00:00:00.000000000
2004-08-15T00:00:00.000000000
2004-08-16T00:00:00.000000000
2004-08-17T00:00:00.000000000
2004-08-18T00:00:00.000000000
2004-08-19T00:00:00.000000000
2004-08-20T00:00:00.000000000
2004-08-21T00:00:00.000000000
2004-08-22T00:00:00.000000000
2004-08-23T00:00:00.000000000
2004-08-24T00:00:00.000000000
2004-08-25T00:00:00.000000000
2004-08-26T00:00:00.000000000
2004-08-27T00:00:00.000000000
2004-08-28T00:00:00.000000000
2004-08-29T00:00:00.000000000
2004-08-30T00:00:00.000000000
2004-08-31

2006-06-27T00:00:00.000000000
2006-06-28T00:00:00.000000000
2006-06-29T00:00:00.000000000
2006-06-30T00:00:00.000000000
2006-07-01T00:00:00.000000000
2006-07-02T00:00:00.000000000
2006-07-03T00:00:00.000000000
2006-07-04T00:00:00.000000000
2006-07-05T00:00:00.000000000
2006-07-06T00:00:00.000000000
2006-07-07T00:00:00.000000000
2006-07-08T00:00:00.000000000
2006-07-09T00:00:00.000000000
2006-07-10T00:00:00.000000000
2006-07-11T00:00:00.000000000
2006-07-12T00:00:00.000000000
2006-07-13T00:00:00.000000000
2006-07-14T00:00:00.000000000
2006-07-15T00:00:00.000000000
2006-07-16T00:00:00.000000000
2006-07-17T00:00:00.000000000
2006-07-18T00:00:00.000000000
2006-07-19T00:00:00.000000000
2006-07-20T00:00:00.000000000
2006-07-21T00:00:00.000000000
2006-07-22T00:00:00.000000000
2006-07-23T00:00:00.000000000
2006-07-24T00:00:00.000000000
2006-07-25T00:00:00.000000000
2006-07-26T00:00:00.000000000
2006-07-27T00:00:00.000000000
2006-07-28T00:00:00.000000000
2006-07-29T00:00:00.000000000
2006-07-30

2008-05-26T00:00:00.000000000
2008-05-27T00:00:00.000000000
2008-05-28T00:00:00.000000000
2008-05-29T00:00:00.000000000
2008-05-30T00:00:00.000000000
2008-05-31T00:00:00.000000000
2008-06-01T00:00:00.000000000
2008-06-02T00:00:00.000000000
2008-06-03T00:00:00.000000000
2008-06-04T00:00:00.000000000
2008-06-05T00:00:00.000000000
2008-06-06T00:00:00.000000000
2008-06-07T00:00:00.000000000
2008-06-08T00:00:00.000000000
2008-06-09T00:00:00.000000000
2008-06-10T00:00:00.000000000
2008-06-11T00:00:00.000000000
2008-06-12T00:00:00.000000000
2008-06-13T00:00:00.000000000
2008-06-14T00:00:00.000000000
2008-06-15T00:00:00.000000000
2008-06-16T00:00:00.000000000
2008-06-17T00:00:00.000000000
2008-06-18T00:00:00.000000000
2008-06-19T00:00:00.000000000
2008-06-20T00:00:00.000000000
2008-06-21T00:00:00.000000000
2008-06-22T00:00:00.000000000
2008-06-23T00:00:00.000000000
2008-06-24T00:00:00.000000000
2008-06-25T00:00:00.000000000
2008-06-26T00:00:00.000000000
2008-06-27T00:00:00.000000000
2008-06-28

...

2011-08-23T00:00:00.000000000
2011-08-24T00:00:00.000000000
2011-08-25T00:00:00.000000000
2011-08-26T00:00:00.000000000
2011-08-27T00:00:00.000000000
2011-08-28T00:00:00.000000000
2011-08-29T00:00:00.000000000
2011-08-30T00:00:00.000000000
2011-08-31T00:00:00.000000000
2011-09-01T00:00:00.000000000
2011-09-02T00:00:00.000000000
2011-09-03T00:00:00.000000000
2011-09-04T00:00:00.000000000
2011-09-05T00:00:00.000000000
2011-09-06T00:00:00.000000000
2011-09-07T00:00:00.000000000
2011-09-08T00:00:00.000000000
2011-09-09T00:00:00.000000000
2011-09-10T00:00:00.000000000
2011-09-11T00:00:00.000000000
2011-09-12T00:00:00.000000000
2011-09-13T00:00:00.000000000
2011-09-14T00:00:00.000000000
2011-09-15T00:00:00.000000000
2011-09-16T00:00:00.000000000
2011-09-17T00:00:00.000000000
2011-09-18T00:00:00.000000000
2011-09-19T00:00:00.000000000
2011-09-20T00:00:00.000000000
2011-09-21T00:00:00.000000000
2011-09-22T00:00:00.000000000
2011-09-23T00:00:00.000000000
2011-09-24T00:00:00.000000000
2011-09-25

2013-07-23T00:00:00.000000000
2013-07-24T00:00:00.000000000
2013-07-25T00:00:00.000000000
2013-07-26T00:00:00.000000000
2013-07-27T00:00:00.000000000
2013-07-28T00:00:00.000000000
2013-07-29T00:00:00.000000000
2013-07-30T00:00:00.000000000
2013-07-31T00:00:00.000000000
2013-08-01T00:00:00.000000000
2013-08-02T00:00:00.000000000
2013-08-03T00:00:00.000000000
2013-08-04T00:00:00.000000000
2013-08-05T00:00:00.000000000
2013-08-06T00:00:00.000000000
2013-08-07T00:00:00.000000000
2013-08-08T00:00:00.000000000
2013-08-09T00:00:00.000000000
2013-08-10T00:00:00.000000000
2013-08-11T00:00:00.000000000
2013-08-12T00:00:00.000000000
2013-08-13T00:00:00.000000000
2013-08-14T00:00:00.000000000
2013-08-15T00:00:00.000000000
2013-08-16T00:00:00.000000000
2013-08-17T00:00:00.000000000
2013-08-18T00:00:00.000000000
2013-08-19T00:00:00.000000000
2013-08-20T00:00:00.000000000
2013-08-21T00:00:00.000000000
2013-08-22T00:00:00.000000000
2013-08-23T00:00:00.000000000
2013-08-24T00:00:00.000000000
2013-08-25

2015-06-22T00:00:00.000000000
2015-06-23T00:00:00.000000000
2015-06-24T00:00:00.000000000
2015-06-25T00:00:00.000000000
2015-06-26T00:00:00.000000000
2015-06-27T00:00:00.000000000
2015-06-28T00:00:00.000000000
2015-06-29T00:00:00.000000000
2015-06-30T00:00:00.000000000
2015-07-01T00:00:00.000000000
2015-07-02T00:00:00.000000000
2015-07-03T00:00:00.000000000
2015-07-04T00:00:00.000000000
2015-07-05T00:00:00.000000000
2015-07-06T00:00:00.000000000
2015-07-07T00:00:00.000000000
2015-07-08T00:00:00.000000000
2015-07-09T00:00:00.000000000
2015-07-10T00:00:00.000000000
2015-07-11T00:00:00.000000000
2015-07-12T00:00:00.000000000
2015-07-13T00:00:00.000000000
2015-07-14T00:00:00.000000000
2015-07-15T00:00:00.000000000
2015-07-16T00:00:00.000000000
2015-07-17T00:00:00.000000000
2015-07-18T00:00:00.000000000
2015-07-19T00:00:00.000000000
2015-07-20T00:00:00.000000000
2015-07-21T00:00:00.000000000
2015-07-22T00:00:00.000000000
2015-07-23T00:00:00.000000000
2015-07-24T00:00:00.000000000
2015-07-25

2017-05-21T00:00:00.000000000
2017-05-22T00:00:00.000000000
2017-05-23T00:00:00.000000000
2017-05-24T00:00:00.000000000
2017-05-25T00:00:00.000000000
2017-05-26T00:00:00.000000000
2017-05-27T00:00:00.000000000
2017-05-28T00:00:00.000000000
2017-05-29T00:00:00.000000000
2017-05-30T00:00:00.000000000
2017-05-31T00:00:00.000000000
2017-06-01T00:00:00.000000000
2017-06-02T00:00:00.000000000
2017-06-03T00:00:00.000000000
2017-06-04T00:00:00.000000000
2017-06-05T00:00:00.000000000
2017-06-06T00:00:00.000000000
2017-06-07T00:00:00.000000000
2017-06-08T00:00:00.000000000
2017-06-09T00:00:00.000000000
2017-06-10T00:00:00.000000000
2017-06-11T00:00:00.000000000
2017-06-12T00:00:00.000000000
2017-06-13T00:00:00.000000000
2017-06-14T00:00:00.000000000
2017-06-15T00:00:00.000000000
2017-06-16T00:00:00.000000000
2017-06-17T00:00:00.000000000
2017-06-18T00:00:00.000000000
2017-06-19T00:00:00.000000000
2017-06-20T00:00:00.000000000
2017-06-21T00:00:00.000000000
2017-06-22T00:00:00.000000000
2017-06-23

2018-09-19T00:00:00.000000000
2018-09-20T00:00:00.000000000
2018-09-21T00:00:00.000000000
2018-09-22T00:00:00.000000000
2018-09-23T00:00:00.000000000
2018-09-24T00:00:00.000000000
2018-09-25T00:00:00.000000000
2018-09-26T00:00:00.000000000
2018-09-27T00:00:00.000000000
2018-09-28T00:00:00.000000000
2018-09-29T00:00:00.000000000
2018-09-30T00:00:00.000000000


In [149]:
# interpolation of u

num_levels = 41 # equivalent to 4000 meters AGL for every gridpoint

num_times = times.shape[0]

interpolated_u = np.full((num_times, num_levels, lats.shape[0], lons.shape[0]), np.nan)

for tidx in range(times.shape[0]):
    
    print(times[tidx])

    for latidx in range(lats.shape[0]):

        for lonidx in range(lons.shape[0]):
            
            u_array = u[tidx, :, latidx, lonidx]

            alt_array = z[tidx, :, latidx, lonidx]

            sfc_alt = surface_altitudes[tidx, latidx, lonidx]

            new_u = interpolate_var_to_altitude(u_array, alt_array, sfc_alt)

            interpolated_u[tidx, :, latidx, lonidx] = new_u

2001-05-01T00:00:00.000000000
2001-05-02T00:00:00.000000000
2001-05-03T00:00:00.000000000
2001-05-04T00:00:00.000000000
2001-05-05T00:00:00.000000000
2001-05-06T00:00:00.000000000
2001-05-07T00:00:00.000000000
2001-05-08T00:00:00.000000000
2001-05-09T00:00:00.000000000
2001-05-10T00:00:00.000000000
2001-05-11T00:00:00.000000000
2001-05-12T00:00:00.000000000
2001-05-13T00:00:00.000000000
2001-05-14T00:00:00.000000000
2001-05-15T00:00:00.000000000
2001-05-16T00:00:00.000000000
2001-05-17T00:00:00.000000000
2001-05-18T00:00:00.000000000
2001-05-19T00:00:00.000000000
2001-05-20T00:00:00.000000000
2001-05-21T00:00:00.000000000
2001-05-22T00:00:00.000000000
2001-05-23T00:00:00.000000000
2001-05-24T00:00:00.000000000
2001-05-25T00:00:00.000000000
2001-05-26T00:00:00.000000000
2001-05-27T00:00:00.000000000
2001-05-28T00:00:00.000000000
2001-05-29T00:00:00.000000000
2001-05-30T00:00:00.000000000
2001-05-31T00:00:00.000000000
2001-06-01T00:00:00.000000000
2001-06-02T00:00:00.000000000
2001-06-03

2002-08-30T00:00:00.000000000
2002-08-31T00:00:00.000000000
2002-09-01T00:00:00.000000000
2002-09-02T00:00:00.000000000
2002-09-03T00:00:00.000000000
2002-09-04T00:00:00.000000000
2002-09-05T00:00:00.000000000
2002-09-06T00:00:00.000000000
2002-09-07T00:00:00.000000000
2002-09-08T00:00:00.000000000
2002-09-09T00:00:00.000000000
2002-09-10T00:00:00.000000000
2002-09-11T00:00:00.000000000
2002-09-12T00:00:00.000000000
2002-09-13T00:00:00.000000000
2002-09-14T00:00:00.000000000
2002-09-15T00:00:00.000000000
2002-09-16T00:00:00.000000000
2002-09-17T00:00:00.000000000
2002-09-18T00:00:00.000000000
2002-09-19T00:00:00.000000000
2002-09-20T00:00:00.000000000
2002-09-21T00:00:00.000000000
2002-09-22T00:00:00.000000000
2002-09-23T00:00:00.000000000
2002-09-24T00:00:00.000000000
2002-09-25T00:00:00.000000000
2002-09-26T00:00:00.000000000
2002-09-27T00:00:00.000000000
2002-09-28T00:00:00.000000000
2002-09-29T00:00:00.000000000
2002-09-30T00:00:00.000000000
2003-05-01T00:00:00.000000000
2003-05-02

2004-07-29T00:00:00.000000000
2004-07-30T00:00:00.000000000
2004-07-31T00:00:00.000000000
2004-08-01T00:00:00.000000000
2004-08-02T00:00:00.000000000
2004-08-03T00:00:00.000000000
2004-08-04T00:00:00.000000000
2004-08-05T00:00:00.000000000
2004-08-06T00:00:00.000000000
2004-08-07T00:00:00.000000000
2004-08-08T00:00:00.000000000
2004-08-09T00:00:00.000000000
2004-08-10T00:00:00.000000000
2004-08-11T00:00:00.000000000
2004-08-12T00:00:00.000000000
2004-08-13T00:00:00.000000000
2004-08-14T00:00:00.000000000
2004-08-15T00:00:00.000000000
2004-08-16T00:00:00.000000000
2004-08-17T00:00:00.000000000
2004-08-18T00:00:00.000000000
2004-08-19T00:00:00.000000000
2004-08-20T00:00:00.000000000
2004-08-21T00:00:00.000000000
2004-08-22T00:00:00.000000000
2004-08-23T00:00:00.000000000
2004-08-24T00:00:00.000000000
2004-08-25T00:00:00.000000000
2004-08-26T00:00:00.000000000
2004-08-27T00:00:00.000000000
2004-08-28T00:00:00.000000000
2004-08-29T00:00:00.000000000
2004-08-30T00:00:00.000000000
2004-08-31

2006-06-27T00:00:00.000000000
2006-06-28T00:00:00.000000000
2006-06-29T00:00:00.000000000
2006-06-30T00:00:00.000000000
2006-07-01T00:00:00.000000000
2006-07-02T00:00:00.000000000
2006-07-03T00:00:00.000000000
2006-07-04T00:00:00.000000000
2006-07-05T00:00:00.000000000
2006-07-06T00:00:00.000000000
2006-07-07T00:00:00.000000000
2006-07-08T00:00:00.000000000
2006-07-09T00:00:00.000000000
2006-07-10T00:00:00.000000000
2006-07-11T00:00:00.000000000
2006-07-12T00:00:00.000000000
2006-07-13T00:00:00.000000000
2006-07-14T00:00:00.000000000
2006-07-15T00:00:00.000000000
2006-07-16T00:00:00.000000000
2006-07-17T00:00:00.000000000
2006-07-18T00:00:00.000000000
2006-07-19T00:00:00.000000000
2006-07-20T00:00:00.000000000
2006-07-21T00:00:00.000000000
2006-07-22T00:00:00.000000000
2006-07-23T00:00:00.000000000
2006-07-24T00:00:00.000000000
2006-07-25T00:00:00.000000000
2006-07-26T00:00:00.000000000
2006-07-27T00:00:00.000000000
2006-07-28T00:00:00.000000000
2006-07-29T00:00:00.000000000
2006-07-30

2008-05-26T00:00:00.000000000
2008-05-27T00:00:00.000000000
2008-05-28T00:00:00.000000000
2008-05-29T00:00:00.000000000
2008-05-30T00:00:00.000000000
2008-05-31T00:00:00.000000000
2008-06-01T00:00:00.000000000
2008-06-02T00:00:00.000000000
2008-06-03T00:00:00.000000000
2008-06-04T00:00:00.000000000
2008-06-05T00:00:00.000000000
2008-06-06T00:00:00.000000000
2008-06-07T00:00:00.000000000
2008-06-08T00:00:00.000000000
2008-06-09T00:00:00.000000000
2008-06-10T00:00:00.000000000
2008-06-11T00:00:00.000000000
2008-06-12T00:00:00.000000000
2008-06-13T00:00:00.000000000
2008-06-14T00:00:00.000000000
2008-06-15T00:00:00.000000000
2008-06-16T00:00:00.000000000
2008-06-17T00:00:00.000000000
2008-06-18T00:00:00.000000000
2008-06-19T00:00:00.000000000
2008-06-20T00:00:00.000000000
2008-06-21T00:00:00.000000000
2008-06-22T00:00:00.000000000
2008-06-23T00:00:00.000000000
2008-06-24T00:00:00.000000000
2008-06-25T00:00:00.000000000
2008-06-26T00:00:00.000000000
2008-06-27T00:00:00.000000000
2008-06-28

...

2011-08-23T00:00:00.000000000
2011-08-24T00:00:00.000000000
2011-08-25T00:00:00.000000000
2011-08-26T00:00:00.000000000
2011-08-27T00:00:00.000000000
2011-08-28T00:00:00.000000000
2011-08-29T00:00:00.000000000
2011-08-30T00:00:00.000000000
2011-08-31T00:00:00.000000000
2011-09-01T00:00:00.000000000
2011-09-02T00:00:00.000000000
2011-09-03T00:00:00.000000000
2011-09-04T00:00:00.000000000
2011-09-05T00:00:00.000000000
2011-09-06T00:00:00.000000000
2011-09-07T00:00:00.000000000
2011-09-08T00:00:00.000000000
2011-09-09T00:00:00.000000000
2011-09-10T00:00:00.000000000
2011-09-11T00:00:00.000000000
2011-09-12T00:00:00.000000000
2011-09-13T00:00:00.000000000
2011-09-14T00:00:00.000000000
2011-09-15T00:00:00.000000000
2011-09-16T00:00:00.000000000
2011-09-17T00:00:00.000000000
2011-09-18T00:00:00.000000000
2011-09-19T00:00:00.000000000
2011-09-20T00:00:00.000000000
2011-09-21T00:00:00.000000000
2011-09-22T00:00:00.000000000
2011-09-23T00:00:00.000000000
2011-09-24T00:00:00.000000000
2011-09-25

2013-07-23T00:00:00.000000000
2013-07-24T00:00:00.000000000
2013-07-25T00:00:00.000000000
2013-07-26T00:00:00.000000000
2013-07-27T00:00:00.000000000
2013-07-28T00:00:00.000000000
2013-07-29T00:00:00.000000000
2013-07-30T00:00:00.000000000
2013-07-31T00:00:00.000000000
2013-08-01T00:00:00.000000000
2013-08-02T00:00:00.000000000
2013-08-03T00:00:00.000000000
2013-08-04T00:00:00.000000000
2013-08-05T00:00:00.000000000
2013-08-06T00:00:00.000000000
2013-08-07T00:00:00.000000000
2013-08-08T00:00:00.000000000
2013-08-09T00:00:00.000000000
2013-08-10T00:00:00.000000000
2013-08-11T00:00:00.000000000
2013-08-12T00:00:00.000000000
2013-08-13T00:00:00.000000000
2013-08-14T00:00:00.000000000
2013-08-15T00:00:00.000000000
2013-08-16T00:00:00.000000000
2013-08-17T00:00:00.000000000
2013-08-18T00:00:00.000000000
2013-08-19T00:00:00.000000000
2013-08-20T00:00:00.000000000
2013-08-21T00:00:00.000000000
2013-08-22T00:00:00.000000000
2013-08-23T00:00:00.000000000
2013-08-24T00:00:00.000000000
2013-08-25

2015-06-21T00:00:00.000000000
2015-06-22T00:00:00.000000000
2015-06-23T00:00:00.000000000
2015-06-24T00:00:00.000000000
2015-06-25T00:00:00.000000000
2015-06-26T00:00:00.000000000
2015-06-27T00:00:00.000000000
2015-06-28T00:00:00.000000000
2015-06-29T00:00:00.000000000
2015-06-30T00:00:00.000000000
2015-07-01T00:00:00.000000000
2015-07-02T00:00:00.000000000
2015-07-03T00:00:00.000000000
2015-07-04T00:00:00.000000000
2015-07-05T00:00:00.000000000
2015-07-06T00:00:00.000000000
2015-07-07T00:00:00.000000000
2015-07-08T00:00:00.000000000
2015-07-09T00:00:00.000000000
2015-07-10T00:00:00.000000000
2015-07-11T00:00:00.000000000
2015-07-12T00:00:00.000000000
2015-07-13T00:00:00.000000000
2015-07-14T00:00:00.000000000
2015-07-15T00:00:00.000000000
2015-07-16T00:00:00.000000000
2015-07-17T00:00:00.000000000
2015-07-18T00:00:00.000000000
2015-07-19T00:00:00.000000000
2015-07-20T00:00:00.000000000
2015-07-21T00:00:00.000000000
2015-07-22T00:00:00.000000000
2015-07-23T00:00:00.000000000
2015-07-24

2017-05-20T00:00:00.000000000
2017-05-21T00:00:00.000000000
2017-05-22T00:00:00.000000000
2017-05-23T00:00:00.000000000
2017-05-24T00:00:00.000000000
2017-05-25T00:00:00.000000000
2017-05-26T00:00:00.000000000
2017-05-27T00:00:00.000000000
2017-05-28T00:00:00.000000000
2017-05-29T00:00:00.000000000
2017-05-30T00:00:00.000000000
2017-05-31T00:00:00.000000000
2017-06-01T00:00:00.000000000
2017-06-02T00:00:00.000000000
2017-06-03T00:00:00.000000000
2017-06-04T00:00:00.000000000
2017-06-05T00:00:00.000000000
2017-06-06T00:00:00.000000000
2017-06-07T00:00:00.000000000
2017-06-08T00:00:00.000000000
2017-06-09T00:00:00.000000000
2017-06-10T00:00:00.000000000
2017-06-11T00:00:00.000000000
2017-06-12T00:00:00.000000000
2017-06-13T00:00:00.000000000
2017-06-14T00:00:00.000000000
2017-06-15T00:00:00.000000000
2017-06-16T00:00:00.000000000
2017-06-17T00:00:00.000000000
2017-06-18T00:00:00.000000000
2017-06-19T00:00:00.000000000
2017-06-20T00:00:00.000000000
2017-06-21T00:00:00.000000000
2017-06-22

2018-09-19T00:00:00.000000000
2018-09-20T00:00:00.000000000
2018-09-21T00:00:00.000000000
2018-09-22T00:00:00.000000000
2018-09-23T00:00:00.000000000
2018-09-24T00:00:00.000000000
2018-09-25T00:00:00.000000000
2018-09-26T00:00:00.000000000
2018-09-27T00:00:00.000000000
2018-09-28T00:00:00.000000000
2018-09-29T00:00:00.000000000
2018-09-30T00:00:00.000000000


In [175]:
# interpolation of v

num_levels = 41 # equivalent to 4000 meters AGL for every gridpoint

num_times = times.shape[0]

interpolated_v = np.full((num_times, num_levels, lats.shape[0], lons.shape[0]), np.nan)

for tidx in range(times.shape[0]):
    
    print(times[tidx])

    for latidx in range(lats.shape[0]):

        for lonidx in range(lons.shape[0]):
            
            v_array = v[tidx, :, latidx, lonidx]

            alt_array = z[tidx, :, latidx, lonidx]

            sfc_alt = surface_altitudes[tidx, latidx, lonidx]

            new_v = interpolate_var_to_altitude(v_array, alt_array, sfc_alt)

            interpolated_v[tidx, :, latidx, lonidx] = new_v

2001-05-01T00:00:00.000000000
2001-05-02T00:00:00.000000000
2001-05-03T00:00:00.000000000
2001-05-04T00:00:00.000000000
2001-05-05T00:00:00.000000000
2001-05-06T00:00:00.000000000
2001-05-07T00:00:00.000000000
2001-05-08T00:00:00.000000000
2001-05-09T00:00:00.000000000
2001-05-10T00:00:00.000000000
2001-05-11T00:00:00.000000000
2001-05-12T00:00:00.000000000
2001-05-13T00:00:00.000000000
2001-05-14T00:00:00.000000000
2001-05-15T00:00:00.000000000
2001-05-16T00:00:00.000000000
2001-05-17T00:00:00.000000000
2001-05-18T00:00:00.000000000
2001-05-19T00:00:00.000000000
2001-05-20T00:00:00.000000000
2001-05-21T00:00:00.000000000
2001-05-22T00:00:00.000000000
2001-05-23T00:00:00.000000000
2001-05-24T00:00:00.000000000
2001-05-25T00:00:00.000000000
2001-05-26T00:00:00.000000000
2001-05-27T00:00:00.000000000
2001-05-28T00:00:00.000000000
2001-05-29T00:00:00.000000000
2001-05-30T00:00:00.000000000
2001-05-31T00:00:00.000000000
2001-06-01T00:00:00.000000000
2001-06-02T00:00:00.000000000
2001-06-03

2002-08-30T00:00:00.000000000
2002-08-31T00:00:00.000000000
2002-09-01T00:00:00.000000000
2002-09-02T00:00:00.000000000
2002-09-03T00:00:00.000000000
2002-09-04T00:00:00.000000000
2002-09-05T00:00:00.000000000
2002-09-06T00:00:00.000000000
2002-09-07T00:00:00.000000000
2002-09-08T00:00:00.000000000
2002-09-09T00:00:00.000000000
2002-09-10T00:00:00.000000000
2002-09-11T00:00:00.000000000
2002-09-12T00:00:00.000000000
2002-09-13T00:00:00.000000000
2002-09-14T00:00:00.000000000
2002-09-15T00:00:00.000000000
2002-09-16T00:00:00.000000000
2002-09-17T00:00:00.000000000
2002-09-18T00:00:00.000000000
2002-09-19T00:00:00.000000000
2002-09-20T00:00:00.000000000
2002-09-21T00:00:00.000000000
2002-09-22T00:00:00.000000000
2002-09-23T00:00:00.000000000
2002-09-24T00:00:00.000000000
2002-09-25T00:00:00.000000000
2002-09-26T00:00:00.000000000
2002-09-27T00:00:00.000000000
2002-09-28T00:00:00.000000000
2002-09-29T00:00:00.000000000
2002-09-30T00:00:00.000000000
2003-05-01T00:00:00.000000000
2003-05-02

2004-07-29T00:00:00.000000000
2004-07-30T00:00:00.000000000
2004-07-31T00:00:00.000000000
2004-08-01T00:00:00.000000000
2004-08-02T00:00:00.000000000
2004-08-03T00:00:00.000000000
2004-08-04T00:00:00.000000000
2004-08-05T00:00:00.000000000
2004-08-06T00:00:00.000000000
2004-08-07T00:00:00.000000000
2004-08-08T00:00:00.000000000
2004-08-09T00:00:00.000000000
2004-08-10T00:00:00.000000000
2004-08-11T00:00:00.000000000
2004-08-12T00:00:00.000000000
2004-08-13T00:00:00.000000000
2004-08-14T00:00:00.000000000
2004-08-15T00:00:00.000000000
2004-08-16T00:00:00.000000000
2004-08-17T00:00:00.000000000
2004-08-18T00:00:00.000000000
2004-08-19T00:00:00.000000000
2004-08-20T00:00:00.000000000
2004-08-21T00:00:00.000000000
2004-08-22T00:00:00.000000000
2004-08-23T00:00:00.000000000
2004-08-24T00:00:00.000000000
2004-08-25T00:00:00.000000000
2004-08-26T00:00:00.000000000
2004-08-27T00:00:00.000000000
2004-08-28T00:00:00.000000000
2004-08-29T00:00:00.000000000
2004-08-30T00:00:00.000000000
2004-08-31

2006-06-28T00:00:00.000000000
2006-06-29T00:00:00.000000000
2006-06-30T00:00:00.000000000
2006-07-01T00:00:00.000000000
2006-07-02T00:00:00.000000000
2006-07-03T00:00:00.000000000
2006-07-04T00:00:00.000000000
2006-07-05T00:00:00.000000000
2006-07-06T00:00:00.000000000
2006-07-07T00:00:00.000000000
2006-07-08T00:00:00.000000000
2006-07-09T00:00:00.000000000
2006-07-10T00:00:00.000000000
2006-07-11T00:00:00.000000000
2006-07-12T00:00:00.000000000
2006-07-13T00:00:00.000000000
2006-07-14T00:00:00.000000000
2006-07-15T00:00:00.000000000
2006-07-16T00:00:00.000000000
2006-07-17T00:00:00.000000000
2006-07-18T00:00:00.000000000
2006-07-19T00:00:00.000000000
2006-07-20T00:00:00.000000000
2006-07-21T00:00:00.000000000
2006-07-22T00:00:00.000000000
2006-07-23T00:00:00.000000000
2006-07-24T00:00:00.000000000
2006-07-25T00:00:00.000000000
2006-07-26T00:00:00.000000000
2006-07-27T00:00:00.000000000
2006-07-28T00:00:00.000000000
2006-07-29T00:00:00.000000000
2006-07-30T00:00:00.000000000
2006-07-31

2008-05-27T00:00:00.000000000
2008-05-28T00:00:00.000000000
2008-05-29T00:00:00.000000000
2008-05-30T00:00:00.000000000
2008-05-31T00:00:00.000000000
2008-06-01T00:00:00.000000000
2008-06-02T00:00:00.000000000
2008-06-03T00:00:00.000000000
2008-06-04T00:00:00.000000000
2008-06-05T00:00:00.000000000
2008-06-06T00:00:00.000000000
2008-06-07T00:00:00.000000000
2008-06-08T00:00:00.000000000
2008-06-09T00:00:00.000000000
2008-06-10T00:00:00.000000000
2008-06-11T00:00:00.000000000
2008-06-12T00:00:00.000000000
2008-06-13T00:00:00.000000000
2008-06-14T00:00:00.000000000
2008-06-15T00:00:00.000000000
2008-06-16T00:00:00.000000000
2008-06-17T00:00:00.000000000
2008-06-18T00:00:00.000000000
2008-06-19T00:00:00.000000000
2008-06-20T00:00:00.000000000
2008-06-21T00:00:00.000000000
2008-06-22T00:00:00.000000000
2008-06-23T00:00:00.000000000
2008-06-24T00:00:00.000000000
2008-06-25T00:00:00.000000000
2008-06-26T00:00:00.000000000
2008-06-27T00:00:00.000000000
2008-06-28T00:00:00.000000000
2008-06-29

...

2011-08-24T00:00:00.000000000
2011-08-25T00:00:00.000000000
2011-08-26T00:00:00.000000000
2011-08-27T00:00:00.000000000
2011-08-28T00:00:00.000000000
2011-08-29T00:00:00.000000000
2011-08-30T00:00:00.000000000
2011-08-31T00:00:00.000000000
2011-09-01T00:00:00.000000000
2011-09-02T00:00:00.000000000
2011-09-03T00:00:00.000000000
2011-09-04T00:00:00.000000000
2011-09-05T00:00:00.000000000
2011-09-06T00:00:00.000000000
2011-09-07T00:00:00.000000000
2011-09-08T00:00:00.000000000
2011-09-09T00:00:00.000000000
2011-09-10T00:00:00.000000000
2011-09-11T00:00:00.000000000
2011-09-12T00:00:00.000000000
2011-09-13T00:00:00.000000000
2011-09-14T00:00:00.000000000
2011-09-15T00:00:00.000000000
2011-09-16T00:00:00.000000000
2011-09-17T00:00:00.000000000
2011-09-18T00:00:00.000000000
2011-09-19T00:00:00.000000000
2011-09-20T00:00:00.000000000
2011-09-21T00:00:00.000000000
2011-09-22T00:00:00.000000000
2011-09-23T00:00:00.000000000
2011-09-24T00:00:00.000000000
2011-09-25T00:00:00.000000000
2011-09-26

2013-07-23T00:00:00.000000000
2013-07-24T00:00:00.000000000
2013-07-25T00:00:00.000000000
2013-07-26T00:00:00.000000000
2013-07-27T00:00:00.000000000
2013-07-28T00:00:00.000000000
2013-07-29T00:00:00.000000000
2013-07-30T00:00:00.000000000
2013-07-31T00:00:00.000000000
2013-08-01T00:00:00.000000000
2013-08-02T00:00:00.000000000
2013-08-03T00:00:00.000000000
2013-08-04T00:00:00.000000000
2013-08-05T00:00:00.000000000
2013-08-06T00:00:00.000000000
2013-08-07T00:00:00.000000000
2013-08-08T00:00:00.000000000
2013-08-09T00:00:00.000000000
2013-08-10T00:00:00.000000000
2013-08-11T00:00:00.000000000
2013-08-12T00:00:00.000000000
2013-08-13T00:00:00.000000000
2013-08-14T00:00:00.000000000
2013-08-15T00:00:00.000000000
2013-08-16T00:00:00.000000000
2013-08-17T00:00:00.000000000
2013-08-18T00:00:00.000000000
2013-08-19T00:00:00.000000000
2013-08-20T00:00:00.000000000
2013-08-21T00:00:00.000000000
2013-08-22T00:00:00.000000000
2013-08-23T00:00:00.000000000
2013-08-24T00:00:00.000000000
2013-08-25

2015-06-22T00:00:00.000000000
2015-06-23T00:00:00.000000000
2015-06-24T00:00:00.000000000
2015-06-25T00:00:00.000000000
2015-06-26T00:00:00.000000000
2015-06-27T00:00:00.000000000
2015-06-28T00:00:00.000000000
2015-06-29T00:00:00.000000000
2015-06-30T00:00:00.000000000
2015-07-01T00:00:00.000000000
2015-07-02T00:00:00.000000000
2015-07-03T00:00:00.000000000
2015-07-04T00:00:00.000000000
2015-07-05T00:00:00.000000000
2015-07-06T00:00:00.000000000
2015-07-07T00:00:00.000000000
2015-07-08T00:00:00.000000000
2015-07-09T00:00:00.000000000
2015-07-10T00:00:00.000000000
2015-07-11T00:00:00.000000000
2015-07-12T00:00:00.000000000
2015-07-13T00:00:00.000000000
2015-07-14T00:00:00.000000000
2015-07-15T00:00:00.000000000
2015-07-16T00:00:00.000000000
2015-07-17T00:00:00.000000000
2015-07-18T00:00:00.000000000
2015-07-19T00:00:00.000000000
2015-07-20T00:00:00.000000000
2015-07-21T00:00:00.000000000
2015-07-22T00:00:00.000000000
2015-07-23T00:00:00.000000000
2015-07-24T00:00:00.000000000
2015-07-25

2017-05-22T00:00:00.000000000
2017-05-23T00:00:00.000000000
2017-05-24T00:00:00.000000000
2017-05-25T00:00:00.000000000
2017-05-26T00:00:00.000000000
2017-05-27T00:00:00.000000000
2017-05-28T00:00:00.000000000
2017-05-29T00:00:00.000000000
2017-05-30T00:00:00.000000000
2017-05-31T00:00:00.000000000
2017-06-01T00:00:00.000000000
2017-06-02T00:00:00.000000000
2017-06-03T00:00:00.000000000
2017-06-04T00:00:00.000000000
2017-06-05T00:00:00.000000000
2017-06-06T00:00:00.000000000
2017-06-07T00:00:00.000000000
2017-06-08T00:00:00.000000000
2017-06-09T00:00:00.000000000
2017-06-10T00:00:00.000000000
2017-06-11T00:00:00.000000000
2017-06-12T00:00:00.000000000
2017-06-13T00:00:00.000000000
2017-06-14T00:00:00.000000000
2017-06-15T00:00:00.000000000
2017-06-16T00:00:00.000000000
2017-06-17T00:00:00.000000000
2017-06-18T00:00:00.000000000
2017-06-19T00:00:00.000000000
2017-06-20T00:00:00.000000000
2017-06-21T00:00:00.000000000
2017-06-22T00:00:00.000000000
2017-06-23T00:00:00.000000000
2017-06-24

2018-09-20T00:00:00.000000000
2018-09-21T00:00:00.000000000
2018-09-22T00:00:00.000000000
2018-09-23T00:00:00.000000000
2018-09-24T00:00:00.000000000
2018-09-25T00:00:00.000000000
2018-09-26T00:00:00.000000000
2018-09-27T00:00:00.000000000
2018-09-28T00:00:00.000000000
2018-09-29T00:00:00.000000000
2018-09-30T00:00:00.000000000


In [184]:
# interpolation of w

num_levels = 41 # equivalent to 4000 meters AGL for every gridpoint

num_times = times.shape[0]

interpolated_w = np.full((num_times, num_levels, lats.shape[0], lons.shape[0]), np.nan)

for tidx in range(times.shape[0]):
    
    print(times[tidx])

    for latidx in range(lats.shape[0]):

        for lonidx in range(lons.shape[0]):
            
            w_array = w[tidx, :, latidx, lonidx]

            alt_array = z[tidx, :, latidx, lonidx]

            sfc_alt = surface_altitudes[tidx, latidx, lonidx]

            new_w = interpolate_var_to_altitude(w_array, alt_array, sfc_alt)

            interpolated_w[tidx, :, latidx, lonidx] = new_w

2001-05-01T00:00:00.000000000
2001-05-02T00:00:00.000000000
2001-05-03T00:00:00.000000000
2001-05-04T00:00:00.000000000
2001-05-05T00:00:00.000000000
2001-05-06T00:00:00.000000000
2001-05-07T00:00:00.000000000
2001-05-08T00:00:00.000000000
2001-05-09T00:00:00.000000000
2001-05-10T00:00:00.000000000
2001-05-11T00:00:00.000000000
2001-05-12T00:00:00.000000000
2001-05-13T00:00:00.000000000
2001-05-14T00:00:00.000000000
2001-05-15T00:00:00.000000000
2001-05-16T00:00:00.000000000
2001-05-17T00:00:00.000000000
2001-05-18T00:00:00.000000000
2001-05-19T00:00:00.000000000
2001-05-20T00:00:00.000000000
2001-05-21T00:00:00.000000000
2001-05-22T00:00:00.000000000
2001-05-23T00:00:00.000000000
2001-05-24T00:00:00.000000000
2001-05-25T00:00:00.000000000
2001-05-26T00:00:00.000000000
2001-05-27T00:00:00.000000000
2001-05-28T00:00:00.000000000
2001-05-29T00:00:00.000000000
2001-05-30T00:00:00.000000000
2001-05-31T00:00:00.000000000
2001-06-01T00:00:00.000000000
2001-06-02T00:00:00.000000000
2001-06-03

2002-08-30T00:00:00.000000000
2002-08-31T00:00:00.000000000
2002-09-01T00:00:00.000000000
2002-09-02T00:00:00.000000000
2002-09-03T00:00:00.000000000
2002-09-04T00:00:00.000000000
2002-09-05T00:00:00.000000000
2002-09-06T00:00:00.000000000
2002-09-07T00:00:00.000000000
2002-09-08T00:00:00.000000000
2002-09-09T00:00:00.000000000
2002-09-10T00:00:00.000000000
2002-09-11T00:00:00.000000000
2002-09-12T00:00:00.000000000
2002-09-13T00:00:00.000000000
2002-09-14T00:00:00.000000000
2002-09-15T00:00:00.000000000
2002-09-16T00:00:00.000000000
2002-09-17T00:00:00.000000000
2002-09-18T00:00:00.000000000
2002-09-19T00:00:00.000000000
2002-09-20T00:00:00.000000000
2002-09-21T00:00:00.000000000
2002-09-22T00:00:00.000000000
2002-09-23T00:00:00.000000000
2002-09-24T00:00:00.000000000
2002-09-25T00:00:00.000000000
2002-09-26T00:00:00.000000000
2002-09-27T00:00:00.000000000
2002-09-28T00:00:00.000000000
2002-09-29T00:00:00.000000000
2002-09-30T00:00:00.000000000
2003-05-01T00:00:00.000000000
2003-05-02

2004-07-29T00:00:00.000000000
2004-07-30T00:00:00.000000000
2004-07-31T00:00:00.000000000
2004-08-01T00:00:00.000000000
2004-08-02T00:00:00.000000000
2004-08-03T00:00:00.000000000
2004-08-04T00:00:00.000000000
2004-08-05T00:00:00.000000000
2004-08-06T00:00:00.000000000
2004-08-07T00:00:00.000000000
2004-08-08T00:00:00.000000000
2004-08-09T00:00:00.000000000
2004-08-10T00:00:00.000000000
2004-08-11T00:00:00.000000000
2004-08-12T00:00:00.000000000
2004-08-13T00:00:00.000000000
2004-08-14T00:00:00.000000000
2004-08-15T00:00:00.000000000
2004-08-16T00:00:00.000000000
2004-08-17T00:00:00.000000000
2004-08-18T00:00:00.000000000
2004-08-19T00:00:00.000000000
2004-08-20T00:00:00.000000000
2004-08-21T00:00:00.000000000
2004-08-22T00:00:00.000000000
2004-08-23T00:00:00.000000000
2004-08-24T00:00:00.000000000
2004-08-25T00:00:00.000000000
2004-08-26T00:00:00.000000000
2004-08-27T00:00:00.000000000
2004-08-28T00:00:00.000000000
2004-08-29T00:00:00.000000000
2004-08-30T00:00:00.000000000
2004-08-31

2006-06-27T00:00:00.000000000
2006-06-28T00:00:00.000000000
2006-06-29T00:00:00.000000000
2006-06-30T00:00:00.000000000
2006-07-01T00:00:00.000000000
2006-07-02T00:00:00.000000000
2006-07-03T00:00:00.000000000
2006-07-04T00:00:00.000000000
2006-07-05T00:00:00.000000000
2006-07-06T00:00:00.000000000
2006-07-07T00:00:00.000000000
2006-07-08T00:00:00.000000000
2006-07-09T00:00:00.000000000
2006-07-10T00:00:00.000000000
2006-07-11T00:00:00.000000000
2006-07-12T00:00:00.000000000
2006-07-13T00:00:00.000000000
2006-07-14T00:00:00.000000000
2006-07-15T00:00:00.000000000
2006-07-16T00:00:00.000000000
2006-07-17T00:00:00.000000000
2006-07-18T00:00:00.000000000
2006-07-19T00:00:00.000000000
2006-07-20T00:00:00.000000000
2006-07-21T00:00:00.000000000
2006-07-22T00:00:00.000000000
2006-07-23T00:00:00.000000000
2006-07-24T00:00:00.000000000
2006-07-25T00:00:00.000000000
2006-07-26T00:00:00.000000000
2006-07-27T00:00:00.000000000
2006-07-28T00:00:00.000000000
2006-07-29T00:00:00.000000000
2006-07-30

2008-05-26T00:00:00.000000000
2008-05-27T00:00:00.000000000
2008-05-28T00:00:00.000000000
2008-05-29T00:00:00.000000000
2008-05-30T00:00:00.000000000
2008-05-31T00:00:00.000000000
2008-06-01T00:00:00.000000000
2008-06-02T00:00:00.000000000
2008-06-03T00:00:00.000000000
2008-06-04T00:00:00.000000000
2008-06-05T00:00:00.000000000
2008-06-06T00:00:00.000000000
2008-06-07T00:00:00.000000000
2008-06-08T00:00:00.000000000
2008-06-09T00:00:00.000000000
2008-06-10T00:00:00.000000000
2008-06-11T00:00:00.000000000
2008-06-12T00:00:00.000000000
2008-06-13T00:00:00.000000000
2008-06-14T00:00:00.000000000
2008-06-15T00:00:00.000000000
2008-06-16T00:00:00.000000000
2008-06-17T00:00:00.000000000
2008-06-18T00:00:00.000000000
2008-06-19T00:00:00.000000000
2008-06-20T00:00:00.000000000
2008-06-21T00:00:00.000000000
2008-06-22T00:00:00.000000000
2008-06-23T00:00:00.000000000
2008-06-24T00:00:00.000000000
2008-06-25T00:00:00.000000000
2008-06-26T00:00:00.000000000
2008-06-27T00:00:00.000000000
2008-06-28

...

2011-08-24T00:00:00.000000000
2011-08-25T00:00:00.000000000
2011-08-26T00:00:00.000000000
2011-08-27T00:00:00.000000000
2011-08-28T00:00:00.000000000
2011-08-29T00:00:00.000000000
2011-08-30T00:00:00.000000000
2011-08-31T00:00:00.000000000
2011-09-01T00:00:00.000000000
2011-09-02T00:00:00.000000000
2011-09-03T00:00:00.000000000
2011-09-04T00:00:00.000000000
2011-09-05T00:00:00.000000000
2011-09-06T00:00:00.000000000
2011-09-07T00:00:00.000000000
2011-09-08T00:00:00.000000000
2011-09-09T00:00:00.000000000
2011-09-10T00:00:00.000000000
2011-09-11T00:00:00.000000000
2011-09-12T00:00:00.000000000
2011-09-13T00:00:00.000000000
2011-09-14T00:00:00.000000000
2011-09-15T00:00:00.000000000
2011-09-16T00:00:00.000000000
2011-09-17T00:00:00.000000000
2011-09-18T00:00:00.000000000
2011-09-19T00:00:00.000000000
2011-09-20T00:00:00.000000000
2011-09-21T00:00:00.000000000
2011-09-22T00:00:00.000000000
2011-09-23T00:00:00.000000000
2011-09-24T00:00:00.000000000
2011-09-25T00:00:00.000000000
2011-09-26

2013-07-23T00:00:00.000000000
2013-07-24T00:00:00.000000000
2013-07-25T00:00:00.000000000
2013-07-26T00:00:00.000000000
2013-07-27T00:00:00.000000000
2013-07-28T00:00:00.000000000
2013-07-29T00:00:00.000000000
2013-07-30T00:00:00.000000000
2013-07-31T00:00:00.000000000
2013-08-01T00:00:00.000000000
2013-08-02T00:00:00.000000000
2013-08-03T00:00:00.000000000
2013-08-04T00:00:00.000000000
2013-08-05T00:00:00.000000000
2013-08-06T00:00:00.000000000
2013-08-07T00:00:00.000000000
2013-08-08T00:00:00.000000000
2013-08-09T00:00:00.000000000
2013-08-10T00:00:00.000000000
2013-08-11T00:00:00.000000000
2013-08-12T00:00:00.000000000
2013-08-13T00:00:00.000000000
2013-08-14T00:00:00.000000000
2013-08-15T00:00:00.000000000
2013-08-16T00:00:00.000000000
2013-08-17T00:00:00.000000000
2013-08-18T00:00:00.000000000
2013-08-19T00:00:00.000000000
2013-08-20T00:00:00.000000000
2013-08-21T00:00:00.000000000
2013-08-22T00:00:00.000000000
2013-08-23T00:00:00.000000000
2013-08-24T00:00:00.000000000
2013-08-25

2015-06-22T00:00:00.000000000
2015-06-23T00:00:00.000000000
2015-06-24T00:00:00.000000000
2015-06-25T00:00:00.000000000
2015-06-26T00:00:00.000000000
2015-06-27T00:00:00.000000000
2015-06-28T00:00:00.000000000
2015-06-29T00:00:00.000000000
2015-06-30T00:00:00.000000000
2015-07-01T00:00:00.000000000
2015-07-02T00:00:00.000000000
2015-07-03T00:00:00.000000000
2015-07-04T00:00:00.000000000
2015-07-05T00:00:00.000000000
2015-07-06T00:00:00.000000000
2015-07-07T00:00:00.000000000
2015-07-08T00:00:00.000000000
2015-07-09T00:00:00.000000000
2015-07-10T00:00:00.000000000
2015-07-11T00:00:00.000000000
2015-07-12T00:00:00.000000000
2015-07-13T00:00:00.000000000
2015-07-14T00:00:00.000000000
2015-07-15T00:00:00.000000000
2015-07-16T00:00:00.000000000
2015-07-17T00:00:00.000000000
2015-07-18T00:00:00.000000000
2015-07-19T00:00:00.000000000
2015-07-20T00:00:00.000000000
2015-07-21T00:00:00.000000000
2015-07-22T00:00:00.000000000
2015-07-23T00:00:00.000000000
2015-07-24T00:00:00.000000000
2015-07-25

2017-05-22T00:00:00.000000000
2017-05-23T00:00:00.000000000
2017-05-24T00:00:00.000000000
2017-05-25T00:00:00.000000000
2017-05-26T00:00:00.000000000
2017-05-27T00:00:00.000000000
2017-05-28T00:00:00.000000000
2017-05-29T00:00:00.000000000
2017-05-30T00:00:00.000000000
2017-05-31T00:00:00.000000000
2017-06-01T00:00:00.000000000
2017-06-02T00:00:00.000000000
2017-06-03T00:00:00.000000000
2017-06-04T00:00:00.000000000
2017-06-05T00:00:00.000000000
2017-06-06T00:00:00.000000000
2017-06-07T00:00:00.000000000
2017-06-08T00:00:00.000000000
2017-06-09T00:00:00.000000000
2017-06-10T00:00:00.000000000
2017-06-11T00:00:00.000000000
2017-06-12T00:00:00.000000000
2017-06-13T00:00:00.000000000
2017-06-14T00:00:00.000000000
2017-06-15T00:00:00.000000000
2017-06-16T00:00:00.000000000
2017-06-17T00:00:00.000000000
2017-06-18T00:00:00.000000000
2017-06-19T00:00:00.000000000
2017-06-20T00:00:00.000000000
2017-06-21T00:00:00.000000000
2017-06-22T00:00:00.000000000
2017-06-23T00:00:00.000000000
2017-06-24

2018-09-21T00:00:00.000000000
2018-09-22T00:00:00.000000000
2018-09-23T00:00:00.000000000
2018-09-24T00:00:00.000000000
2018-09-25T00:00:00.000000000
2018-09-26T00:00:00.000000000
2018-09-27T00:00:00.000000000
2018-09-28T00:00:00.000000000
2018-09-29T00:00:00.000000000
2018-09-30T00:00:00.000000000


In [188]:
# Step 1: Define the backup directory in your home folder

new_alt = np.arange(0, 4100, 100)

backup_dir = os.path.expanduser("~/backup_data")  # Expands ~ to full home directory path

os.makedirs(backup_dir, exist_ok=True)  # create the directory if it doesn't exist

# Step 3: Save multiple arrays in a compressed file

file_path = os.path.join(backup_dir, "backup_arrays_for_advection.npz")  # Full path to save file

np.savez_compressed(file_path,
                    
                    times=times, 
                    
                    interpolated_alt=new_alt,
                    
                    lats=lats, 
                    
                    lons=lons,
                    
                    interpolated_pressure=interpolated_pressure, 
                    
                    interpolated_q=interpolated_q, 
                    
                    interpolated_u=interpolated_u,
                    
                    interpolated_v=interpolated_v, 
                    
                    interpolated_w=interpolated_w)

print(f"Arrays saved to {file_path}")

# Step 4: Verify that the file exists
print("Saved files:", os.listdir(backup_dir))


Arrays saved to /home/annierosen16/backup_data/backup_arrays_for_advection.npz
Saved files: ['backup_arrays_for_buoyancy.npz', 'backup_metadata_for_buoyancy.npz', 'ctp_hilow_input_arrays.npz', 'backup_arrays_for_advection.npz']
